In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 
import sys

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()

from rowgenerators import geoframe, parse_app_url


In [4]:
pkg = mp.jupyter.open_package()
#pkg = mp.jupyter.open_source_package()
pkg

# Child and Adult Care Food Program (CACFP) Sites
`cde.ca.gov-cacfp_sites-2` Last Update: 2019-06-18T04:33:10

_The Child and Adult Care Food Program (CACFP) provides federal funding to participating child care and adult day care centers, allowing the centers to serve nutritious meals to enrolled participants_


California Department of Education CACFP sites file, with extra features,
include:

* Sites geocoded to Lat/Lon and Census tract
* The `program_types` field is broken out into dummy variables
* CDS codes are matched from the CDE schools list. 

## CDS Code Matching

There are 5 different methods for matching sites to schools in the schools
file, each with its own code:

* `fhs`: Fuzzy hash string, a has string based n the address. 
* `fhs_nz`: The fuzzy hash strin, but without the zip code, which occasionally is wrong or changes. 
* `name_city`: A lightly normalized ( alpha only, lowercase, combination of the site name and the site city
* `name_county`: A lightly normalized combination of the site name and county
* `mp_name`: The site name and county, with each word converted to it's metaphone. 

The matching method code is included in the `match_type` columns. For many of
the matching types there can be more than one CDS code associated with the
matching string, particularly the strings based on addresses. The ``match_qc``
value is the inverse of the number of values with the same matching code, so if
the algorithm selected the first of 5 CDS codes, the ``match_qc`` value is .2

Only about of the 35% of the 5076 sites are matched. The number matched for each type of matching tecchnique is: 

    Not Matched    3292
    fhs            1602
    name_city       161
    name_county      19
    mp_name           1
    fhs_nz            1
    
## Versions

1. Initial version
2. Add faked CDS codes for sites that are not schools
## Documentation Links

* [Program home page](https://www.cde.ca.gov/ls/nu/cc/index.asp) 
* [Download page](https://www.cde.ca.gov/ds/sh/sn/cacfpcentermap.asp) 

 
 
## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)
 
## Resources

* ** [cacfp_sites](data/cacfp_sites.csv)**. CACFP sites, with location geocode and program type dummy variables
 
## References

* **[site_geocodes](data/geocodes.csv)**. Externally geocoded site locations. Geocoded with Geocodio.io
* **[cacfp_sites_source](https://www.cde.ca.gov/ds/sh/sn/documents/apcacfpcentersites2019.xlsx#&headers=4&start=5)**. List of  Child and Adult Care Food Program (CACFP) sites
* **[schools](metapack+http://library.metatab.org/cde.ca.gov-schools-1.csv#public_schools)**. California schools and districts
* **[ca_county_codes](metapack+http://library.metatab.org/sandiegodata.org-county_codes-ca-1.csv#ca_county_codes)**. California government codes for counties, used in  CDS codes

In [7]:
pkg.resource('cacfp_sites')

Header,Type,Description
name,text,
county,string,
address,text,
city,text,
state,string,
zip,string,
geoid,string,Geoid of the Census tract that contains the site
lat,number,Latitude of the site
lon,number,Longitude of the site
sponsor_name,string,


In [5]:
sites = pkg.resource('cacfp_sites').dataframe()

In [6]:
sites.head()

,name,county,address,city,state,zip,geoid,lat,lon,sponsor_name,...,hom,hsc,itc,other,ccc,sac,sps,cdscode,match_type,match_qc
0,24 HOUR OAKLAND DAY CARE CTR,ALAMEDA,4700 EAST 14TH STREET,OAKLAND,CA,94601,14000US06001407400,37.770622,-122.210436,24-HR OAKLAND PARENT-TEACHER CHILDRENS CENTER,...,0,0,0,0,1,0,0,01CACFP0000001,NaN,NaN
1,24 HOUR OAKLAND DAY CARE CTR,ALAMEDA,3500 EAST 9TH ST,OAKLAND,CA,94601,14000US06001406100,37.772727,-122.225031,24-HR OAKLAND PARENT-TEACHER CHILDRENS CENTER,...,0,0,0,0,1,0,0,01CACFP0000002,NaN,NaN
2,ALAMEDA HEAD START - SUE MATHESON INFANT CENTER,ALAMEDA,670 WEST MIDWAY,ALAMEDA,CA,94501,14000US06001428700,37.785245,-122.299103,ALAMEDA FAMILY SERVICES,...,0,0,1,0,1,0,0,01CACFP0000003,NaN,NaN
3,ALAMEDA HS/AGUILAR,ALAMEDA,1901 THIRD STREET,ALAMEDA,CA,94501,14000US06001427600,37.778734,-122.288263,ALAMEDA FAMILY SERVICES,...,0,1,0,0,0,0,0,01CACFP0000004,NaN,NaN
4,ALAMEDA HS/EHS-COLLEGE OF ALAMEDA,ALAMEDA,555 RALPH M. APPEZATO WAY,ALAMEDA,CA,94501,14000US06001427800,37.770563,-122.264779,ALAMEDA FAMILY SERVICES,...,0,1,0,0,0,0,0,01CACFP0000005,NaN,NaN


In [15]:
sites[list(sites.columns)[16:23]].sum()

hom        29
hsc      1402
itc       690
other      11
ccc      1972
sac       714
sps       430
dtype: int64